In [1]:
from petrify.space import Vector, Point, Plane
from petrify import tau, plane

# Extrusion

Groups of [`PlanarPolygon`](https://petrify.readthedocs.io/en/latest/petrify.space.html#petrify.space.PlanarPolygon)s are often the building block for forming complex solids. The most general version of this is the [`Extrusion`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Extrusion) solid. Its input is an array of [`PlanarPolygon`](https://petrify.readthedocs.io/en/latest/petrify.space.html#petrify.space.PlanarPolygon) with the same number of points on each "slice":

In [2]:
from petrify.solid import Extrusion, PlanarPolygon, Basis
square = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(0, 1),
    plane.Point(1, 1),
    plane.Point(1, 0)
])

dz = Vector(0, 0, 1)
tornado = Extrusion([
    PlanarPolygon(Basis.xy + dz * ix, square * (ix / 4) * plane.Matrix.rotate(tau / 16 * ix)) 
    for ix in range(8)
])

tornado.construction().render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

This class constructs rings to connect polygons with the same number of points, and generates endcaps to make the resultant solid "watertight":

In [3]:
tornado.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

### Covering Basis

Polygons defined on the plane require a mapping between their two-dimensional planar point and the point they occupy in three-dimensional space. This transformation is defined via the [`Basis`](https://petrify.readthedocs.io/en/latest/petrify.space.html#petrify.space.Basis) object:

In [4]:
from petrify.solid import Collection
shifted = (square * 2) + plane.Vector(1, 1)

Collection([
    PlanarPolygon(Basis.xy, shifted),
    PlanarPolygon(Basis.xz, shifted),
    PlanarPolygon(Basis.yz, shifted),
]).render(camera={'position': Point(6, 6, 6)})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

This allows for flexibility when defining the "starting point" for extrusion operations:

In [5]:
from petrify.solid import PolygonExtrusion
Collection([
    PolygonExtrusion(PlanarPolygon(Basis.xy, shifted), Vector.basis.z * 0.25),
    PolygonExtrusion(PlanarPolygon(Basis.xz, shifted), Vector.basis.y * 0.25),
    PolygonExtrusion(PlanarPolygon(Basis.yz, shifted), Vector.basis.x * 0.25)
]).render(camera={'position': Point(6, 6, 6)})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

# Spinning

Another neat trick for creating objects with rotational symmetry is the [`Spun`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Spun) solid. It takes an array of [`Polygon`](https://petrify.readthedocs.io/en/latest/petrify.space.html#petrify.space.Polygon) profiles with the same number of points, and "spins" them, equally rotating each in turn about an axis.

In [6]:
from petrify.solid import Spun

notch = plane.Polygon([
    plane.Point(0, 0),
    plane.Point(1, 0),
    plane.Point(1, 1),
    plane.Point(2, 2),
    plane.Point(1, 3),
    plane.Point(1, 4),
    plane.Point(0, 4)
])

axis = Vector.basis.z
start = Vector.basis.x
notched = Spun(axis, start, [notch] * 10)

notched.construction().render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

The spun profiles are connected along the axis of rotation, and any necessary endcaps are created:

In [7]:
notched.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

### Turning Screws

The true power of [`Spun`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Spun) is on display when each of the segments has a different polygon. In this case, it's critical to understand that the final profile shares the same rotation angle as the starting profile. Thus, the two shapes must match for the solid to remain watertight.

When matched properly to the starting profile, proper matching allows the creation of knurled or threaded objects:

In [8]:
from petrify.solid import Spun

def thread(dy, match=True):
    # thread matching
    start = 0 if dy == 0 and match else 1
    end = 0 if dy == 1 and match else 1

    dy = plane.Vector(0, dy * 2)
    turn = plane.Vector(0, 2)
    return plane.Polygon([
        plane.Point(0, 0),
        plane.Point(1, 0),
        plane.Point(1, 1) + dy,
        plane.Point(1 + start, 2) + dy,
        plane.Point(1, 3) + dy,
        plane.Point(1, 1) + dy + turn,
        plane.Point(1 + end, 2) + dy + turn,
        plane.Point(1, 3) + dy + turn,
        plane.Point(1, 8),
        plane.Point(0, 8)
    ])

segments = 20
screw = Spun(axis, start, [thread(dy / segments) for dy in range(segments + 1)])
screw.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

The code right after the `# thread matching` comment is used to start and end the threads. It is thus critical for forming a proper solid. Here's what happens without it:

In [9]:
screw = Spun(axis, start, [thread(dy / segments, match=False) for dy in range(segments + 1)])
screw.render(camera={'position': Point(8, 8, 4)})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…